# 데이터 전처리(Preprocessing)

spacy 라이브러리: 문장의 토큰화, 태깅 등의 전처리 기능을 위한 라이브러리

영어와 독일어 전처리 모듈 설치

In [1]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [2]:
import spacy

In [3]:
spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

In [4]:
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i} : {token.text}")

인덱스 0 : I
인덱스 1 : am
인덱스 2 : a
인덱스 3 : graduate
인덱스 4 : student
인덱스 5 : .


## 토큰화 함수 정의 (spacy의 토크나이저 이용)

In [5]:
# 독일어 문장을 토큰화 하는 함수
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]


In [6]:
import torch
print(torch.__version__)

2.1.0+cu121


In [7]:
!pip install torchtext==0.16.0
!pip install portalocker>=2.0.0

## 어휘집 Vocab 만들기

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

In [9]:
SRC_LANG = 'de'
TGT_LANG = 'en'

In [10]:
tokenizer = {}
tokenizer['en'] = tokenize_en
tokenizer['de'] = tokenize_de

In [11]:
# 토큰 목록을 생성하기 위한 헬퍼(helper) 함수
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANG: 0, TGT_LANG: 1}

    for i, datasample_tuple in enumerate(train_iter):
      yield tokenizer[language](datasample_tuple[language_index[language]])
    '''
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])
    '''

In [12]:
# 특수 기호(symbol)와 인덱스를 정의합니다
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# 토큰들이 어휘집(vocab)에 인덱스 순서대로 잘 삽입되어 있는지 확인합니다
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

vocab_transform = {} # 영어, 독일어에 대해서 torchtext의 Vocab 옵젝이 저장됨.

for ln in [SRC_LANG, TGT_LANG]:
  train_iter = Multi30k(split='train', language_pair=(SRC_LANG, TGT_LANG))
  val_iter = Multi30k(split='valid', language_pair=(SRC_LANG, TGT_LANG))
  test_iter = Multi30k(split='test', language_pair=(SRC_LANG, TGT_LANG))

  vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                  min_freq=2, # 최소 2번 이상 등장한 단어만을 선택
                                                  specials=special_symbols,
                                                  special_first=True)

In [13]:
# ``UNK_IDX`` 를 기본 인덱스로 설정합니다. 이 인덱스는 토큰을 찾지 못하는 경우에 반환됩니다.
# 만약 기본 인덱스를 설정하지 않으면 어휘집(Vocabulary)에서 토큰을 찾지 못하는 경우
# ``RuntimeError`` 가 발생합니다.
for ln in [SRC_LANG, TGT_LANG]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [14]:
print('소스 언어의 Vocab(어휘집)')
for idx in range(20):
  print(f'[Vocab] index: {idx} | token: {vocab_transform[SRC_LANG].lookup_token(idx)}')

소스 언어의 Vocab(어휘집)
[Vocab] index: 0 | token: <unk>
[Vocab] index: 1 | token: <pad>
[Vocab] index: 2 | token: <bos>
[Vocab] index: 3 | token: <eos>
[Vocab] index: 4 | token: .
[Vocab] index: 5 | token: Ein
[Vocab] index: 6 | token: einem
[Vocab] index: 7 | token: in
[Vocab] index: 8 | token: ,
[Vocab] index: 9 | token: und
[Vocab] index: 10 | token: mit
[Vocab] index: 11 | token: auf
[Vocab] index: 12 | token: Mann
[Vocab] index: 13 | token: einer
[Vocab] index: 14 | token: Eine
[Vocab] index: 15 | token: ein
[Vocab] index: 16 | token: der
[Vocab] index: 17 | token: Frau
[Vocab] index: 18 | token: eine
[Vocab] index: 19 | token: die


In [15]:
print('타겟 언어의 Vocab(어휘집)')
for idx in range(20):
  print(f'[Vocab] index: {idx} | token: {vocab_transform[TGT_LANG].lookup_token(idx)}')

타겟 언어의 Vocab(어휘집)
[Vocab] index: 0 | token: <unk>
[Vocab] index: 1 | token: <pad>
[Vocab] index: 2 | token: <bos>
[Vocab] index: 3 | token: <eos>
[Vocab] index: 4 | token: a
[Vocab] index: 5 | token: .
[Vocab] index: 6 | token: A
[Vocab] index: 7 | token: in
[Vocab] index: 8 | token: the
[Vocab] index: 9 | token: on
[Vocab] index: 10 | token: is
[Vocab] index: 11 | token: and
[Vocab] index: 12 | token: man
[Vocab] index: 13 | token: of
[Vocab] index: 14 | token: with
[Vocab] index: 15 | token: ,
[Vocab] index: 16 | token: woman
[Vocab] index: 17 | token: are
[Vocab] index: 18 | token: to
[Vocab] index: 19 | token: Two


In [22]:
print(len(vocab_transform[SRC_LANG]))
print(len(vocab_transform[TGT_LANG]))

8014
6191


# Transformer 활용 Seq2Seq 모델

torch에서 제공하는 Transformer을 사용하지 않고, transformer을 구현하여 활용해보자.

In [16]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
BATCH_SIZE = 128

## Multi Head Attention

어텐션은 세가지 요소를 입력으로 받는다.
- 쿼리(queries)
- 키(keys)
- 값(values)

하이퍼 파라미터
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- n_heads: 헤드의 개수(scaled dot-product attention 개수)
- dropout_ratio: 드롭아웃 비율

In [23]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    # assert는 뒤의 조건이 True가 아니면 AssertError를 발생한다.
    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim # 임베딩 차원
    self.n_heads = n_heads # 헤드의 개수(서로 다른 어텐션 컨셉의 수)
    self.head_dim = hidden_dim // n_heads # 각 헤드에서의 임베딩 차원 = 전체 임베딩 차원을 헤드의 수로 나눈 값

    self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
    self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
    self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

    self.fc_o = nn.Linear(hidden_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):
    batch_size = query.shape[0]

    # query: [batch_size, query_len, hidden_dim]
    # key: [batch_size, key_len, hidden_dim]
    # value: [batch_size, value_len, hidden_dim]

    # 각각 FC 레이어에 입력
    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    # Q: [batch_size, query_len, hidden_dim]
    # K: [batch_size, key_len, hidden_dim]
    # V: [batch_size, value_len, hidden_dim]

    # hidden_dim -> n_heads X head_dim 형태로 변형
    # after permute
    # Q: [batch_size, query_len, n_heads, head_dim] -> [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, key_len, n_heads, head_dim] -> [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, value_len, n_heads, head_dim] -> [batch_size, n_heads, value_len, head_dim]
    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

    # Q: [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]

    # Attention Energy 계산 (유사도 계산)
    energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

    # 마스크를 사용할 경우
    if mask is not None:
      energy = energy.masked_fill(mask==0, -1e10) # 마스크 값이 0인 부분에 상당이 작은 값으로 채워준다.

    # 어텐션 스코어 계산: 각 단어에 대한 확률 값
    attention = torch.softmax(energy, dim=-1) # 소프트맥스로 정규화

    # attention: [batch_size, n_heads, query_len, key_len]

    # Scaled Dot-Product Attention을 계산
    x = torch.matmul(self.dropout(attention), V)

    # x: [batch_size, n_heads, query_len, head_dim]

    x = x.permute(0,2,1,3).contiguous()

    # x: [batch_size, query_len, n_heads, head_dim]

    # n_heads X head_dim -> hidden_dim 변형
    x = x.view(batch_size, -1, self.hidden_dim)

    # x: [batch_size, query_len, hidden_dim]

    x = self.fc_o(x)

    return x, attention
